In [1]:
### 
from agents import Agent, Runner
from IPython.display import display, Markdown
import os
from openai import OpenAI
from bs4 import BeautifulSoup
API_KEY = ''
os.environ['OPENAI_API_KEY']=API_KEY
client = OpenAI()

In [37]:
import os
import time
import json
import re
import requests
import pandas as pd
from typing import TypedDict, List, Dict, Any
from bs4 import BeautifulSoup
from geofetch import Geofetcher
from openai import OpenAI
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda

# ----------------------------------------
# Agent state schema
# ----------------------------------------

class AgentState(TypedDict):
    query_gene: str
    pathway_info: Dict[str, Any]
    metadata: Dict[str, Any]
    gse_list: List[str]
    research_plan: str

# ----------------------------------------
# Utility functions (LLM + GEO search)
# ----------------------------------------

client = OpenAI()

def extract_json_block(text):
    match = re.search(r"```json\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return json.loads(match.group(1))
    else:
        return json.loads(text)

def generate_pathway_info(query_gene, model="gpt-4o-mini", temperature=0.3):
    prompt = f"""
You are a biomedical assistant.

Given the gene {query_gene}, return:
1. Key gene symbols in the same biological pathway.
2. Drugs or compounds that inhibit this pathway or {query_gene}'s activity.
3. The disease areas or biological processes this pathway is involved in.

Provide answers in JSON format with keys: "genes", "drugs", "pathways"
    """
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}]
    )
    content = completion.choices[0].message.content
    return extract_json_block(content)

def scrape_organism_from_geo_html(geo_accession):
    url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={geo_accession}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        organism_row = soup.find("td", string="Organism")
        if organism_row and organism_row.find_next_sibling("td"):
            return organism_row.find_next_sibling("td").text.strip()
    except Exception as e:
        print(f"⚠️ Failed to scrape organism for {geo_accession}: {e}")
    return "Unknown"

def search_geo_datasets(keyword, retmax=10):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "gds",
        "term": keyword,
        "retmode": "json",
        "retmax": retmax
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json().get("esearchresult", {}).get("idlist", [])

def fetch_gse_accessions(id_list):
    if not id_list:
        return {}
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    params = {
        "db": "gds",
        "id": ",".join(id_list),
        "retmode": "json"
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    summaries = response.json().get("result", {})
    summaries.pop("uids", None)
    gse_dict = {}
    for uid, info in summaries.items():
        accession = info.get("accession")
        title = info.get("title")
        link = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={accession}"
        organism = scrape_organism_from_geo_html(accession)
        gse_dict[accession] = {
            "title": title,
            "organism": organism,
            "link": link
        }
    return gse_dict

def search_geo_items(item_list, label="gene", max_results=5):
    results = {}
    for item in item_list:
        query = f"{item} AND rna-seq"
        print(f"🔍 Searching GEO for {label}: {item}")
        ids = search_geo_datasets(query, retmax=max_results)
        datasets = fetch_gse_accessions(ids)
        results[item] = datasets
        time.sleep(0.3)
    return results

def get_geofetch_projects(gse_list, metadata_folder="geofetch_metadata"):
    geof = Geofetcher(
        processed=True,
        acc_anno=True,
        discard_soft=True,
        metadata_folder=metadata_folder
    )
    projects = {}
    for gse in gse_list:
        try:
            print(f"📥 Fetching metadata for {gse}")
            result = geof.get_projects(gse)
            projects.update(result)
        except Exception as e:
            print(f"❌ Failed to fetch {gse}: {e}")
    return projects

import subprocess
def download_processed_files_via_cli(gse_list, output_dir="geofetch_metadata", overwrite=False):
    for gse in gse_list:
        gse_path = os.path.join(output_dir, gse)
        if os.path.exists(gse_path) and not overwrite:
            print(f"✅ {gse}: already exists at {gse_path}, skipping.")
            continue
        try:
            subprocess.run(
                ["geofetch", "-i", gse, "--processed", "-m", output_dir],
                check=True
            )
            print(f"✅ Finished downloading for {gse}")
        except subprocess.CalledProcessError as e:
            print(f"❌ geofetch failed for {gse}: {e}")

def save_combined_metadata_csv_from_state(state: Dict, csv_path: str = "geofetch_metadata/combined_metadata.csv") -> pd.DataFrame:
    metadata = state.get("metadata", {})
    if not metadata:
        raise ValueError("No metadata found in the agent state.")

    all_dfs = []
    for gse, project in metadata.items():
        try:
            df = project.sample_table.copy()
            df["source_gse"] = gse
            all_dfs.append(df)
        except Exception as e:
            print(f"⚠️ Failed to extract sample_table from {gse}: {e}")

    if not all_dfs:
        raise ValueError("No sample tables to save.")

    combined_df = pd.concat(all_dfs, ignore_index=True)
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    combined_df.to_csv(csv_path, index=False)
    print(f"✅ Combined metadata saved to: {os.path.abspath(csv_path)}")
    return combined_df

# ----------------------------------------
# Agent 1: Ingestor node
# ----------------------------------------

def ingest_and_prepare(state: dict) -> dict:
    query_gene = state["query_gene"]
    pathway_info = generate_pathway_info(query_gene)
    gene_list = pathway_info.get("genes", [])
    drug_list = pathway_info.get("drugs", [])

    gene_results = search_geo_items(gene_list, label="gene")
    drug_results = search_geo_items(drug_list, label="drug")

    all_gse = set()
    for r in [gene_results, drug_results]:
        for v in r.values():
            all_gse.update(v.keys())
    all_gse = list(all_gse)

    metadata = get_geofetch_projects(all_gse, metadata_folder="geofetch_metadata")
    download_processed_files_via_cli(all_gse, output_dir="geofetch_metadata")

    try:
        save_combined_metadata_csv_from_state({"metadata": metadata})
    except Exception as e:
        print(f"⚠️ Failed to save metadata CSV: {e}")

    return {
        "query_gene": query_gene,
        "pathway_info": pathway_info,
        "metadata": metadata,
        "gse_list": all_gse
    }

# ----------------------------------------
# Agent 2: Analyst node
# ----------------------------------------

def analyze_metadata_and_plan(state: AgentState) -> AgentState:
    metadata = state["metadata"]
    drug_list = state["pathway_info"].get("drugs", [])
    query_gene = state["query_gene"]
    selected = []

    for gse, project in metadata.items():
        df = project.sample_table
        if "processed_file_ftp" in df.columns and df["processed_file_ftp"].notna().any():
            if any(drug.lower() in df.to_string().lower() for drug in drug_list):
                selected.append((gse, df.shape[0]))

    plan = f"🧬 Research Plan for {query_gene} and drugs {drug_list}:\n"
    if not selected:
        plan += "No relevant processed datasets were found.\n"
    else:
        plan += f"{len(selected)} datasets selected:\n"
        for gse, n in selected:
            plan += f"  - {gse} ({n} samples)\n"
        plan += "\nNext: perform differential expression and gene signature clustering."

    return {**state, "research_plan": plan}


def analyze_metadata_and_plan(state: AgentState) -> AgentState:
    import pandas as pd
    import os

    csv_path = "geofetch_metadata/combined_metadata.csv"
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"❌ Metadata CSV not found at {csv_path}")

    df = pd.read_csv(csv_path)

    # Ensure required columns exist
    required_columns = {"sample_name", "sample_source_name_ch1", "sample_title"}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"❌ Metadata CSV must include columns: {required_columns}")

    # Take first 50 rows for LLM context
    selected_df = df[list(required_columns)].fillna("").head(50)
    table_preview = selected_df.to_markdown(index=False)

    query_gene = state["query_gene"]
    drug_list = state["pathway_info"].get("drugs", [])
    selected_df = df[["sample_name", "sample_source_name_ch1", "sample_title"]].fillna("").head(50)
    table_preview = selected_df.to_markdown(index=False)
    print("🧪 Table preview sent to LLM:\n", table_preview)  # ✅ Add this line

    # Build LLM prompt
    prompt = f"""
You are a biomedical research assistant.

The target gene is **{query_gene}**, and the related drugs of interest are: {', '.join(drug_list)}.

Below is a preview of sample metadata (first 50 rows) from multiple GEO studies. Each row includes:
- sample name
- sample source (cell line, tissue)
- sample title (may indicate treatment or condition)

Your task:
1. Identify which studies include drug-treated samples.
2. Identify the control groups if available.
3. Determine the sample types (e.g., cell lines or tissues).
4. Recommend studies and sample comparisons suitable for differential gene expression and drug-response signature analysis.

Respond with:
- GSE or study names (if known)
- The experimental comparison design
- Why the dataset is suitable (or not)
- Bullet points summarizing each recommended comparison

Sample Metadata Table:

"""

    # Call LLM
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[{"role": "user", "content": prompt}]
    )

    research_plan = response.choices[0].message.content.strip()

    return {
        **state,
        "research_plan": research_plan
    }

###
def analyze_metadata_and_plan(state: AgentState) -> AgentState:
    import pandas as pd
    import os

    csv_path = "geofetch_metadata/combined_metadata.csv"
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"❌ Metadata CSV not found at {csv_path}")

    df = pd.read_csv(csv_path)

    # Ensure required columns are available
    required_cols = ["gse", "sample_name", "sample_title", "sample_source_name_ch1", "sample_geo_accession"]
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"❌ Metadata CSV must contain the following columns: {required_cols}")

    # Clean and preview first 30 rows
    preview_df = df[required_cols].fillna("").head(30)
    preview_text = preview_df.to_string(index=False)

    query_gene = state["query_gene"]
    drug_list = state["pathway_info"].get("drugs", [])

    prompt = f"""
You are a biomedical research assistant.

The target gene is **{query_gene}** and the related drugs of interest are: {', '.join(drug_list)}.

Below is a preview of sample metadata from several GEO datasets.
Each row includes:
- GSE accession
- Sample name
- Sample title (may contain treatment or control info)
- Sample source (cell type or tissue)
- Sample GEO accession

Sample Metadata Table:
{preview_text}

Based on the sample names, titles, and sources:
1. Which GSE studies contain drug-treated samples and matching control groups?
2. What cell types or tissues are used?
3. Which treatments are applied? What are the controls?
4. Recommend GSEs and sample pairs suitable for differential gene expression to identify drug-response gene signatures.

Be specific, refer to GSE and sample names where possible, and explain why you recommend them.
"""

    # Call OpenAI LLM
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[{"role": "user", "content": prompt}]
    )

    research_plan = response.choices[0].message.content.strip()

    return {
        **state,
        "research_plan": research_plan
    }

# ----------------------------------------
# LangGraph pipeline
# ----------------------------------------

workflow = StateGraph(state_schema=AgentState)
workflow.add_node("Ingestor", RunnableLambda(ingest_and_prepare))
workflow.add_node("Analyst", RunnableLambda(analyze_metadata_and_plan))
workflow.set_entry_point("Ingestor")
workflow.add_edge("Ingestor", "Analyst")
workflow.set_finish_point("Analyst")
graph = workflow.compile()

# ----------------------------------------
# Invoke the graph
# ----------------------------------------



In [38]:
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass


In [39]:
if __name__ == "__main__":
    result = graph.invoke({"query_gene": "COX1"})
    print("\n📋 Final Research Plan:")
    print(result["research_plan"])

🔍 Searching GEO for gene: PTGS2
🔍 Searching GEO for gene: PTGS1
🔍 Searching GEO for gene: ALOX5
🔍 Searching GEO for gene: CYP2C9
🔍 Searching GEO for gene: CYP2C19
🔍 Searching GEO for drug: Aspirin
🔍 Searching GEO for drug: Ibuprofen
🔍 Searching GEO for drug: Naproxen
🔍 Searching GEO for drug: Celecoxib
🔍 Searching GEO for drug: Diclofenac


[INFO] [23:48:44] Metadata folder: C:\Users\difen\POPPER\geofetch_metadata\project_name
[INFO] [23:48:44] Trying GSE120596 (not a file) as accession...
[INFO] [23:48:44] Processing accession 1 of 1: 'GSE120596'
[INFO] [23:48:44] Trying GSE120596 (not a file) as accession...
[INFO] [23:48:44] Skipped 0 accessions. Starting now.
[INFO] [23:48:44] Processing accession 1 of 1: 'GSE120596'


📥 Fetching metadata for GSE120596


[INFO] [23:48:53] Total number of processed SERIES files found is: 2
[INFO] [23:48:53] Expanding metadata list...
[INFO] [23:48:53] Expanding metadata list...
[INFO] [23:48:53] Finished processing 1 accession(s)
[INFO] [23:48:53] Cleaning soft files ...
[INFO] [23:48:53] No files found. No data to save. File geofetch_metadata\project_name\GSE120596_samples\GSE120596_samples.csv won't be created
[INFO] [23:48:53] Trying GSE156453 (not a file) as accession...
[INFO] [23:48:53] Trying GSE156453 (not a file) as accession...
[INFO] [23:48:53] Skipped 0 accessions. Starting now.
[INFO] [23:48:53] Processing accession 1 of 1: 'GSE156453'


📥 Fetching metadata for GSE156453


...

📥 Fetching metadata for GSE95802


[INFO] [23:59:12] Total number of processed SERIES files found is: 1
[INFO] [23:59:12] Expanding metadata list...
[INFO] [23:59:12] Expanding metadata list...
[INFO] [23:59:12] Finished processing 1 accession(s)
[INFO] [23:59:12] Cleaning soft files ...
[INFO] [23:59:12] No files found. No data to save. File geofetch_metadata\project_name\GSE95802_samples\GSE95802_samples.csv won't be created
[INFO] [23:59:12] Trying GSE139045 (not a file) as accession...
[INFO] [23:59:12] Trying GSE139045 (not a file) as accession...
[INFO] [23:59:12] Skipped 0 accessions. Starting now.
[INFO] [23:59:12] Processing accession 1 of 1: 'GSE139045'


📥 Fetching metadata for GSE139045


[INFO] [23:59:14] Total number of processed SERIES files found is: 1
[INFO] [23:59:14] Expanding metadata list...
[INFO] [23:59:14] Expanding metadata list...
[INFO] [23:59:14] Finished processing 1 accession(s)
[INFO] [23:59:14] Cleaning soft files ...
[INFO] [23:59:14] No files found. No data to save. File geofetch_metadata\project_name\GSE139045_samples\GSE139045_samples.csv won't be created


✅ GSE120596: already exists at geofetch_metadata\GSE120596, skipping.
✅ GSE156453: already exists at geofetch_metadata\GSE156453, skipping.
✅ GSE279800: already exists at geofetch_metadata\GSE279800, skipping.
✅ GSE242272: already exists at geofetch_metadata\GSE242272, skipping.
✅ GSE139044: already exists at geofetch_metadata\GSE139044, skipping.
✅ GSE124074: already exists at geofetch_metadata\GSE124074, skipping.
✅ GSE245768: already exists at geofetch_metadata\GSE245768, skipping.
✅ GSE180857: already exists at geofetch_metadata\GSE180857, skipping.
✅ GSE131732: already exists at geofetch_metadata\GSE131732, skipping.
✅ GSE242369: already exists at geofetch_metadata\GSE242369, skipping.
✅ GSE110282: already exists at geofetch_metadata\GSE110282, skipping.
✅ GSE278083: already exists at geofetch_metadata\GSE278083, skipping.
✅ GSE175744: already exists at geofetch_metadata\GSE175744, skipping.
✅ GSE184884: already exists at geofetch_metadata\GSE184884, skipping.
✅ GSE38809: already 

In [34]:
os.path.exists("geofetch_metadata/combined_metadata.csv")

True